In [ ]:
"""
Источник: https://github.com/NVIDIA/NeMo/blob/main/tutorials/asr/Online_Noise_Augmentation.ipynb
"""

## Установите зависимости
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install text-unidecode

# ## Установите NeMo
BRANCH = 'main'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[asr]

## Установите TorchAudio
!pip install torchaudio>=0.13.0 -f https://download.pytorch.org/whl/torch_stable.html

## Возьмите конфигурацию, которую мы будем использовать в этом примере
!mkdir configs
!pip install librosa==0.9.2


In [ ]:
import json
import librosa
import os
import subprocess
import glob
import torch
import IPython.display as ipd
import soundfile as sf
# Импорт компонента дополнения данных из коллекции ASR
from nemo.collections.asr.parts.preprocessing import perturb, segment

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Введение

Аугументация данных является полезным методом для улучшения производительности моделей, который применим в различных областях. Некоторые дополнения могут также существенно улучшить устойчивость моделей к зашумленным выборкам. 

В этом блокноте мы описываем, как построить пайплайн аугументаций внутри [Neural Modules (NeMo)] (https://github.com/NVIDIA/NeMo).

В блокноте будут описаны следующие шаги:

 - Подготовка набора данных: Подготовка набора данных шума с использованием файла примера.

 - Построение пайплайна аугументаций данных.

## Примечание
Аугументация данных важна для многих наборов данных, но это происходит за счет увеличения времени обучения, если выборки дополняются во время обучения. Некоторые аугументации являются особенно дорогостоящими с точки зрения того, сколько времени они занимают для обработки одной выборки. Вот несколько примеров медленных аугментаций, доступных в NeMo: 

 - Возмущение скорости
 - Возмущение временной растяжки (уровень образца)
 - Шумовое возмущение
 - Импульсное возмущение
 - Аугментация растяжения времени (пакетный уровень, нейронный модуль)
 
Для таких дополнений рекомендуется предварительно обработать набор данных в автономном режиме за единовременную стоимость предварительной обработки, а затем обучить набор данных на этом дополненном наборе.

In [ ]:
# Измените это, если вы не хотите, чтобы данные извлекались в текущий каталог.
data_dir = '.'

In [ ]:
for audio_path in glob.glob("/content/drive/MyDrive/SPbU_smart-assistant/raw/*.*"):
  sample, sr = librosa.core.load(audio_path)
  sf.write(audio_path, sample, samplerate=sr)

In [ ]:
# Загрузите набор данных. Это займет несколько мгновений...
print("******")
if not os.path.exists("./raw"):
  os.makedirs("./raw")
for audio_path in glob.glob("/content/drive/MyDrive/SPbU_smart-assistant/raw/*.*"):
  !cp -v $audio_path "./raw"
print("Конец.\n******")

# Пайплайн дополнения данных

Построение пайплайна дополнения данных в NeMo так же просто, как составление вложенного словаря, который описывает две вещи: 

1) Вероятность возникновения этого дополнения - с помощью ключевого слова `prob` <br>
2) Аргументы ключевых слов, необходимые для данного класса дополнений.

Ниже мы покажем несколько примеров таких дополнений. Обратите внимание, что для того, чтобы отличить исходный образец от возмущенного, мы значительно преувеличиваем силу возмущения.

## Подготовка аудиофайла

In [ ]:
# Посмотрим доступные возмущения
perturb.perturbation_types

{'speed': nemo.collections.asr.parts.preprocessing.perturb.SpeedPerturbation,
 'time_stretch': nemo.collections.asr.parts.preprocessing.perturb.TimeStretchPerturbation,
 'gain': nemo.collections.asr.parts.preprocessing.perturb.GainPerturbation,
 'silence': nemo.collections.asr.parts.preprocessing.perturb.SilencePerturbation,
 'impulse': nemo.collections.asr.parts.preprocessing.perturb.ImpulsePerturbation,
 'shift': nemo.collections.asr.parts.preprocessing.perturb.ShiftPerturbation,
 'noise': nemo.collections.asr.parts.preprocessing.perturb.NoisePerturbation,
 'white_noise': nemo.collections.asr.parts.preprocessing.perturb.WhiteNoisePerturbation,
 'rir_noise_aug': nemo.collections.asr.parts.preprocessing.perturb.RirAndNoisePerturbation,
 'transcode_aug': nemo.collections.asr.parts.preprocessing.perturb.TranscodePerturbation,
 'random_segment': nemo.collections.asr.parts.preprocessing.perturb.RandomSegmentPerturbation}

### Получение исходного аудиофайла

In [ ]:
example = "/content/raw/Audio1.wav"
sample, sr = librosa.core.load(example)
ipd.Audio(sample, rate=sr)

### Конвертировать в формат WAV

In [ ]:
import soundfile as sf
sf.write(example, sample, samplerate=sr)

In [ ]:
sample, sr = librosa.core.load(example)
ipd.Audio(sample, rate=sr)

In [ ]:
# NeMo имеет свой собственный класс поддержки для загрузки wav-файлов
def load_audio(filepath) -> segment.AudioSegment:
    sample_segment = segment.AudioSegment.from_file(filepath, target_sr=sr)
    return sample_segment

sample_segment = load_audio(example)
ipd.Audio(sample_segment.samples, rate=sr)

## Возмущение белого шума

Возмущение белого шума выполняется следующими шагами: <br>
1) Случайная выборка амплитуды шума из равномерно распределенного диапазона (определенного в дБ) <br>
2) Выборка гауссовского шума (mean  = 0, std  = 1) с той же длиной, что и аудиосигнал.<br>
3) Масштабируйте этот гауссов шум по амплитуде (в дБ) <br>
4) Добавьте этот вектор шума к исходному образцу.

Примечательно, что исходный сигнал не должен иметь "шипящего звука", постоянно присутствующего в возмущенной версии.

In [ ]:
white_noise = perturb.WhiteNoisePerturbation(min_level=-50, max_level=-30)

# Perturb the audio file
sample_segment = load_audio(example)
white_noise.perturb(sample_segment)
ipd.Audio(sample_segment.samples, rate=sr)

## Возмущения, зависящие от данных

Некоторые возмущения требуют внешнего источника данных для того, чтобы возмутить исходную выборку. Возмущение шумом - прекрасный пример одного из таких возмущений, для которого требуется внешний набор данных с источником шума, чтобы возмутить исходные данные.

### Подготовка манифеста "шумовых" образцов

In [ ]:
# Lets prepare a manifest file using the baseline file itself, cut into 1 second segments

def write_manifest(filepath, data_dir='.', manifest_name='noise_manifest', duration_max=1e9, duration_stride=1.0):
              
    with open(os.path.join(data_dir, manifest_name + '.json'), 'w') as fout:
        y, sr = librosa.load(filepath)
        duration = librosa.get_duration(y=y, sr=sr)

        offsets = []
        durations = []

        if duration > duration_max:
            current_offset = 0.0

            while current_offset < duration:
                difference = duration - current_offset
                segment_duration = min(duration_max, difference)

                offsets.append(current_offset)
                durations.append(segment_duration)

                current_offset += duration_stride

        else:
            offsets.append(0.0)
            durations.append(duration)


        for duration, offset in zip(durations, offsets):
            metadata = {
                'audio_filepath': filepath,
                'duration': duration,
                'label': 'noise',
                'text': '_',  # for compatibility with ASRAudioText collection
                'offset': offset,
            }

            json.dump(metadata, fout)
            fout.write('\n')
            fout.flush()

        name = noise_path.split('/')[-1]
        print(f"Wrote {len(durations)} segments for filename {name}")
            
    print("Finished preparing manifest !")

In [ ]:
# Write a "noise" manifest file
if not os.path.exists("./noises"):
  os.makedirs("./noises")
for audio_path in glob.glob("/content/drive/MyDrive/SPbU_smart-assistant/noises/*.*"):
  !cp -v $audio_path "./noises"

'/content/drive/MyDrive/SPbU_smart-assistant/noises/1.wav' -> './noises/1.wav'
'/content/drive/MyDrive/SPbU_smart-assistant/noises/2.wav' -> './noises/2.wav'
'/content/drive/MyDrive/SPbU_smart-assistant/noises/3.wav' -> './noises/3.wav'
'/content/drive/MyDrive/SPbU_smart-assistant/noises/4.wav' -> './noises/4.wav'
'/content/drive/MyDrive/SPbU_smart-assistant/noises/5.wav' -> './noises/5.wav'
'/content/drive/MyDrive/SPbU_smart-assistant/noises/6.wav' -> './noises/6.wav'
'/content/drive/MyDrive/SPbU_smart-assistant/noises/7.wav' -> './noises/7.wav'


In [ ]:
if not os.path.exists("./manifests"):
  os.makedirs("./manifests")

for noise_path in glob.glob("/content/drive/MyDrive/SPbU_smart-assistant/noises/*.*"):
  noise_sample, noise_sr = librosa.core.load(noise_path)
  sf.write(noise_path, noise_sample, samplerate=noise_sr)
  noise_sample, noise_sr = librosa.core.load(noise_path)
  name = noise_path.split('/')[-1][-5]
  write_manifest(noise_path, data_dir='./manifests', manifest_name=f'noise_{name}s', duration_stride=1.0, duration_max=30.0)

In [ ]:
# Давайте прочитаем файл манифеста шума
noise_manifest_path = '/content/manifests/noise_7s.json'

!head -n 5 {noise_manifest_path}

## Возмущение шума

Возмущение шума выполняется следующими шагами : <br>
1) Случайная выборка шкалы амплитуды образца шума из равномерно распределенного диапазона (определенного в дБ) <br>
2) Случайно выбирается аудиоклип из набора доступных образцов шума <br>
3) Вычислите коэффициент усиления (в дБ), необходимый для шумового клипа по сравнению с исходным образцом, и масштабируйте шум по этому коэффициенту.<br>
4) Если длительность шумового фрагмента меньше, чем длительность исходного аудио, то случайным образом выберите индекс во времени из исходного образца, куда будет добавлен шумовой фрагмент.<br>
5) Если вместо этого шумовой фрагмент длиннее, чем длительность исходного аудио, то произвольно выделите шумовой фрагмент и добавьте полный фрагмент к исходному аудио. <br>

Примечательно, что образец с шумовым возмущением должен звучать так, как будто одновременно воспроизводятся два звука (наложение звука) по сравнению с исходным сигналом. Величина шума зависит от шага (3), а место добавления шума - от шагов (4) и (5).

In [ ]:
import random
rng = 42 #заметим, что вы можете использовать целое число в качестве случайной затравки для воспроизведения результата 
noise = perturb.NoisePerturbation(manifest_path=noise_manifest_path,
                                  min_snr_db=-10, max_snr_db=-10,
                                  max_gain_db=300.0, rng=rng)

# Возмутить аудиофайл
sample_segment = load_audio(example)
noise.perturb(sample_segment)

ipd.Audio(sample_segment.samples, rate=sr)

## RIR и шумовое возмущение
Увеличение RIR с помощью аддитивного шума переднего плана и фона.
В этой реализации аудиоданные дополняются сначала сверткой аудио с импульсной характеристикой помещения
а затем добавляется шум переднего плана и фоновый шум при различных значениях SNR. RIR, шумы переднего плана и фоновые шумы
должны поставляться либо в виде файла манифеста, либо в виде аудиофайлов в формате tarred (быстрее).

### Подготовьте данные rir и манифест

In [ ]:
url = f"https://raw.githubusercontent.com/NVIDIA/NeMo/stable/scripts/dataset_processing/get_openslr_rir_data.py"
!wget --no-cache --backups=1 {url}

In [ ]:
# Это место, куда будут загружены данные rir.
# Измените это, если вы не хотите, чтобы данные были извлечены в текущий каталог.
rir_data_path = '.'
!python get_openslr_rir_data.py --data_root {rir_data_path}
rir_manifest_path = os.path.join(rir_data_path, 'processed', 'rir.json')
!head -n 3 {rir_manifest_path}

### Создать экземпляр RIR

In [ ]:
rir = perturb.RirAndNoisePerturbation(rir_manifest_path=rir_manifest_path, 
                                      rir_prob=1,
                                      noise_manifest_paths=[noise_manifest_path], # используйте путь noise_manifest_path из предыдущего шага
                                      bg_noise_manifest_paths=[noise_manifest_path],
                                      min_snr_db=[20],# шум переднего плана snr
                                      max_snr_db=[20],
                                      bg_min_snr_db=[20], # фоновый шум snr
                                      bg_max_snr_db=[20],
                                      noise_tar_filepaths=[None],# `[None]` указывает, что шумовые аудиофайлы не являются tar.
                                      bg_noise_tar_filepaths=[None])

### Возмущение звука

In [ ]:
sample_segment = load_audio(example)
rir.perturb(sample_segment)
ipd.Audio(sample_segment.samples, rate=sr)

## Возмущение скорости

Возмущение скорости изменяет скорость речи, но не сохраняет высоту тона звука. Попробуйте несколько случайных возмущений, чтобы увидеть, как меняется высота тона при изменении длительности аудиофайла.

**Примечание**: Это очень медленная аугментация, и ее не рекомендуется выполнять в режиме онлайн для больших наборов данных, так как это может значительно увеличить время обучения.

In [ ]:
resample_type = 'kaiser_best'  # Может быть ['kaiser_best', 'kaiser_fast', 'fft', 'scipy']
speed = perturb.SpeedPerturbation(sr, resample_type, min_speed_rate=0.5, max_speed_rate=2.0, num_rates=-1)

# Возмутить аудиофайл
sample_segment = load_audio(example)
speed.perturb(sample_segment)

ipd.Audio(sample_segment.samples, rate=sr)

## Возмущение временной растяжки

Возмущение Time Stretch изменяет скорость речи, а также сохраняет высоту тона звука. <br>
Попробуйте несколько случайных возмущений, чтобы увидеть, как высота тона остается почти неизменной при изменении длительности аудиофайла.

### Примечание об оптимизации скорости

Растягивание времени является дорогостоящим дополнением и может привести к резкому увеличению времени обучения. Предлагается установить библиотеку `numba` с помощью conda, чтобы использовать более оптимизированное ядро аугментации.

```python
conda install numba
```

In [ ]:
time_stretch = perturb.TimeStretchPerturbation(min_speed_rate=0.7, max_speed_rate=1.5, num_rates=3)

# Возмутить аудиофайл
sample_segment = load_audio(example)
time_stretch.perturb(sample_segment)

ipd.Audio(sample_segment.samples, rate=sr)

##ImpulsePerturbation
Convolves audio with a Room Impulse Response. <br>
Args:
*   manifest_path (list): Manifest file for RIRs
*   shift_impulse (bool): Shift impulse response to adjust for delay at the beginning


        


In [ ]:
impulse = perturb.ImpulsePerturbation(manifest_path=noise_manifest_path,shift_impulse=True)

sample_segment = load_audio(example)
impulse.perturb(sample_segment)
ipd.Audio(sample_segment.samples, rate=sr)

# Конвейер дополнений

Конвейер дополнений может быть построен несколькими способами, либо явно путем инстанцирования объектов этих возмущений, либо неявно путем предоставления аргументов этих дополнений в виде вложенного словаря.

Мы покажем оба подхода в следующих разделах

In [ ]:
perturb.perturbation_types  # Доступные возмущения

### Устанавливаем возмущения

In [ ]:
perturbations = [
    perturb.WhiteNoisePerturbation(min_level=-70, max_level=-45),
    perturb.GainPerturbation(min_gain_dbfs=0, max_gain_dbfs=300),
    perturb.NoisePerturbation(manifest_path=noise_manifest_path,
                                min_snr_db=20, max_snr_db=10, max_gain_db=300.0)
]

### Выберите вероятность применения возмущений

In [ ]:
probas = [1.0, 1.0, 1.0]

### Подготовьте объект аудио дополнения

In [ ]:
augmentations = list(zip(probas, perturbations))
audio_augmentations = perturb.AudioAugmentor(augmentations)

audio_augmentations._pipeline

In [ ]:
sample_segment = load_audio(example)
audio_augmentations.perturb(sample_segment)
ipd.Audio(sample_segment.samples, rate=sr)

In [ ]:
path = '/content/raw/Audio2.wav'
sample_segment = load_audio(path)
audio_augmentations.perturb(sample_segment)
ipd.Audio(sample_segment.samples, rate=sr)

In [ ]:
probas = [1.0, 1.0, 1.0]
for i in range(14):
  perturbations = [
    perturb.WhiteNoisePerturbation(min_level=-80, max_level=-40),
    perturb.GainPerturbation(min_gain_dbfs=0, max_gain_dbfs=30),
    perturb.NoisePerturbation(manifest_path=f'./manifests/noise_{i%7+1}s.json',
                              min_snr_db=20, max_snr_db=10, max_gain_db=30.0)]

  augmentations = list(zip(probas, perturbations))
  audio_augmentations = perturb.AudioAugmentor(augmentations)

  for path in glob.glob("./raw/*.*"):
    sample_segment = load_audio(path)
    audio_augmentations.perturb(sample_segment)
    name = path.split('/')[-1][:-4]

    sf.write(f"./drive/MyDrive/SPbU_smart-assistant/auto/{name}_{i}.wav", 
             sample_segment.samples, samplerate=sr, subtype='PCM_24')